In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import timeit

In [4]:
import random
import math

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
trainData = pd.read_csv("kddTrain.csv")
testData = pd.read_csv("kddTest.csv")

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
stdScaler = StandardScaler()

In [11]:
stdScaler.fit(trainData.drop('41', axis=1))
X_train = stdScaler.transform(trainData.drop('41', axis=1))

In [12]:
stdScaler.fit(testData.drop('41', axis=1))
X_test = stdScaler.transform(testData.drop('41', axis=1))

In [13]:
y_train = trainData['41']

In [14]:
y_test = testData['41']

In [27]:
startTime = timeit.default_timer()
knn = KNeighborsClassifier(n_neighbors= 1, n_jobs=-1)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
elapsedTime = timeit.default_timer() - startTime
print("elapsed time : ",elapsedTime)

elapsed time :  1792.3811873049126


In [28]:
print(confusion_matrix(y_test, predictions))
print ('\n')
print(classification_report(y_test, predictions))

[[ 25099    954]
 [   385 138236]]


             precision    recall  f1-score   support

          0       0.98      0.96      0.97     26053
          1       0.99      1.00      1.00    138621

avg / total       0.99      0.99      0.99    164674



In [23]:
#SCA IMPLEMENTATION
#begin

print("hello")
errorRate = []
totalFeatures = len(X_train[0])
a = 1
r1 = a
t = 0
subsets = []    
error = 0
trainedFeatures = []
#SETTING RANDOM SEED FOR REPRODUCABLE RESULTS
random.seed(14)
#Randomly initialization of each search agents in the population X(t = 1,2,...,m)
subsets.append(random.sample(range(1,totalFeatures),int(totalFeatures/3)))#initializing first random subset
lowestError = 100 #initializing as highest errorRate for comparison purpose


#Function for checking the new generated subset tempSet is already present in the already initialized subsets or not
def isRepeat(presentSubsets, newSubset):
    #print(subsets)
    #print(tempSet)
    newSubset.sort()
    for subset in presentSubsets:
        subset.sort()
        if(subset[:] == newSubset[:]):
            return True
    return False
        
#for generating random search agents in the population   
def randomSearchAgents(m, n):
    i = 1
    while i<m:
        X = random.sample(range(1,totalFeatures),n)
        if (isRepeat(subsets, X) == False):
            subsets.append(X)
            i += 1
        else:
            continue
            
#EQUATION 1
def updateUsingSineCosine(X):
    featureSubset = []
    for j in range(len(X)):
        #print(X[j])
        if(s%2 == 0):
            feature = int(X[j] + r1*math.sin(r2)*abs(r3*P[j]-X[j]))
        else:
             feature = int(X[j] + r1*math.cos(r2)*abs(r3*P[j]-X[j]))
        feature %= totalFeatures
        featureSubset.append(feature)
    return featureSubset


#FOR CHECKING THE SCORE FOR EACH SELECTED FEATURE SUBSET
def trainTestScore(X_train, y_train, X_test, y_test):
    startTime = timeit.default_timer()
    knn = KNeighborsClassifier(n_neighbors = 1, n_jobs=-1)
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_test)
    elapsedTime = timeit.default_timer() - startTime
    print("time taken : ",elapsedTime)
    error = np.mean(prediction != y_test)
    errorRate.append(error)
    return error

#here subsets list contains the list of randomly initialized search agent indices

#INITIALIZE the max numbers of iteration Tmax, we can take it as the total number of columns present
Tmax = totalFeatures
P = []

'''randomly initialize feature subsets, parameter1 denotes total number of feature subsets, here taken as total no of feature avaiable
and parameter 2 denotes number of features in each subsets, here we are considering one third of the total number of features'''

randomSearchAgents(totalFeatures, int(totalFeatures/3))

print("randomly initialized subsets: ",subsets)

trainedFeatures = subsets[:]



while t<Tmax:
    #foreach search agent Xi in the population do
    '''NOVEL 
        here the feature subsets generated by randomSearchAgents method are not repeatative, 
        if any randomly generated feature subset is already present in the previous generated 
        ones then it is neglected and a new feature subset is generated randomly and checked again.
        This process continues until required number of feature subsets are generated which are all unique
    NOVEL'''
    for subset in subsets:
        #Evaluate Xi via the fitness function. if f(Xi)
        if(t > 0):
            if(isRepeat(trainedFeatures, subset) == False):
                error = trainTestScore(X_train[:,subset], y_train, X_test[:, subset], y_test)
                trainedFeatures.append(subset)
                #print(trainedFeatures)
                print('new feature : ',subset, "t=",t)
            else:
                print("repeat")
        else:
            error = trainTestScore(X_train[:,subset], y_train, X_test[:, subset], y_test)
        print("subset : ",subset," error : ",error, " lowest error : ", lowestError)
        #if f(Xi) better than f(P) then
        #Set P = Xi;
        if (error < lowestError):
            P = subset
            lowestError = error

    #Update r1 using equation 2

    r1 = a - t*(a/Tmax)#equation 2

    #Generate randomly new values for r2, r3 and s.

    r2 = random.uniform(0.1, 1.0)
    r3 = random.uniform(0.1, 1.0)
    print('\n\n r2:',r2,' r3:',r3,' r1:',r1)
    #foreach search agent Xi in the population do
    #Update Xi using equation 1.
    for i in range(len(subsets)):
        subsets[i] = updateUsingSineCosine(subsets[i])
    print(subsets)
    print('\n\n\n\nIteration number: ',t,'\n\n\n\n')
    t += 1
#Return P best solution obtained so far.
print("best solution: ",P)
print(lowestError)

hello
randomly initialized subsets:  [[5, 7, 13, 15, 16, 17, 18, 19, 20, 22, 28, 29, 34], [6, 8, 10, 15, 17, 18, 21, 22, 23, 24, 26, 29, 38], [1, 2, 5, 6, 7, 8, 11, 13, 18, 22, 28, 29, 40], [7, 8, 9, 12, 13, 15, 20, 24, 29, 32, 34, 38, 40], [1, 4, 6, 7, 8, 17, 18, 19, 24, 32, 33, 34, 38], [6, 8, 10, 11, 14, 15, 16, 25, 27, 28, 30, 31, 34], [3, 5, 7, 16, 17, 19, 26, 27, 29, 31, 32, 34, 39], [2, 9, 12, 16, 18, 23, 24, 26, 28, 29, 31, 37, 39], [2, 3, 4, 7, 10, 13, 17, 20, 25, 31, 33, 34, 36], [1, 2, 5, 7, 8, 17, 19, 23, 25, 33, 34, 38, 40], [1, 10, 12, 14, 15, 16, 17, 22, 24, 25, 33, 35, 38], [3, 11, 13, 16, 18, 24, 25, 29, 31, 32, 33, 36, 39], [3, 4, 5, 6, 7, 16, 17, 21, 22, 26, 28, 31, 32], [4, 9, 10, 14, 17, 23, 25, 26, 29, 32, 34, 36, 40], [5, 6, 9, 10, 15, 24, 26, 27, 28, 29, 33, 35, 40], [2, 3, 11, 15, 18, 23, 27, 28, 30, 31, 34, 36, 37], [1, 3, 4, 5, 7, 8, 11, 22, 24, 26, 28, 35, 39], [1, 3, 9, 10, 17, 20, 23, 25, 29, 30, 32, 35, 36], [1, 4, 6, 16, 18, 19, 23, 28, 29, 30, 33, 35, 3

KeyboardInterrupt: 

In [24]:
P

[2, 3, 11, 15, 18, 23, 27, 28, 30, 31, 34, 36, 37]

In [29]:
startTime = timeit.default_timer()
knn = KNeighborsClassifier(n_neighbors = 1, n_jobs=-1)
knn.fit(X_train[:,P], y_train)
prediction = knn.predict(X_test[:,P])
elapsedTime = timeit.default_timer() - startTime
print("time taken : ",elapsedTime)
error = np.mean(prediction != y_test)
print("error : ",error)

time taken :  830.0018733505858
error :  0.033162490739278815


In [30]:
print(confusion_matrix(y_test, prediction))
print ('\n')
print(classification_report(y_test, prediction))

[[ 23544   2509]
 [  2952 135669]]


             precision    recall  f1-score   support

          0       0.89      0.90      0.90     26053
          1       0.98      0.98      0.98    138621

avg / total       0.97      0.97      0.97    164674



<h5>We have considered only some of the feature subsets here, because considering all fo